## <b>Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
sns.set()

## <b>Import Data

In [1]:
df=pd.read_csv("C:/Users/Souhar/OneDrive/Desktop/Christ programming/MDS372(Machine Learning)/Final Project/Dataset/Food wastage data.csv")

NameError: name 'pd' is not defined

In [ ]:
df=df.drop("Valid",axis=1)

## <b>Data Preprocessing

### <b>Encoding categorical variables

In [ ]:
df["Food Category"].unique()

array(['Fruits & Vegetables', 'Beverages', 'Prepared Food', 'Frozen Food',
       'Meat & Seafood', 'Grains & Cereals', 'Bakery Items',
       'Dairy Products'], dtype=object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse_output=False)  # Updated parameter name

encoded_array = onehot_encoder.fit_transform(df[["Food Category"]])
encoded_df = pd.DataFrame(encoded_array, columns=onehot_encoder.get_feature_names_out(["Food Category"]))

df = pd.concat([df, encoded_df], axis=1)
df=df.drop("Food Category",axis=1)


### <b>Feature Engineering for classification task

In [ ]:
df["Waste Category"] = pd.qcut(df["Total Waste (Tons)"], q=3, labels=["Low", "Medium", "High"])


<b>Encoding Response variable

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

waste_category_order = [["Low", "Medium", "High"]]

encoder = OrdinalEncoder(categories=waste_category_order)
df["Waste Category Encoded"] = encoder.fit_transform(df[["Waste Category"]])
df=df.drop(["Waste Category"],axis=1)
df.head()


,Country,Year,Total Waste (Tons),Economic Loss (Million $),Avg Waste per Capita (Kg),Population (Million),Household Waste (%),Food Category_Bakery Items,Food Category_Beverages,Food Category_Dairy Products,Food Category_Frozen Food,Food Category_Fruits & Vegetables,Food Category_Grains & Cereals,Food Category_Meat & Seafood,Food Category_Prepared Food,Waste Category Encoded
0,France,2023,46299.69,40551.22,120.19,953.05,31.91,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
1,France,2023,33096.57,36980.82,104.74,1105.47,36.06,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,India,2024,11962.89,11196.04,136.21,1311.91,37.09,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Germany,2024,45038.72,39191.25,179.27,1349.45,68.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
4,South Africa,2024,39478.70,40087.26,155.55,375.84,31.16,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


## <b>Feature Importance and Selection

In [ ]:
X=df.drop(["Waste Category Encoded","Country","Total Waste (Tons)"],axis=1)
y=df["Waste Category Encoded"]

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

rfe = RFE(estimator=model, n_features_to_select=14) 
rfe.fit(X, y)  

feature_importances = rfe.estimator_.feature_importances_
selected_features = X.columns[rfe.support_]  

# Print selected features and their importance scores
for feature, importance in zip(selected_features, feature_importances):
    print(f"{feature}: {importance:.4f}")


c:\Users\Souhar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_rfe.py:300: UserWarning: Found n_features_to_select=14 > n_features=13. There will be no feature selection and all features will be kept.
  warnings.warn(


Year: 0.0146
Economic Loss (Million $): 0.7564
Avg Waste per Capita (Kg): 0.0672
Population (Million): 0.0662
Household Waste (%): 0.0648
Food Category_Bakery Items: 0.0040
Food Category_Beverages: 0.0039
Food Category_Dairy Products: 0.0037
Food Category_Frozen Food: 0.0036
Food Category_Fruits & Vegetables: 0.0045
Food Category_Grains & Cereals: 0.0036
Food Category_Meat & Seafood: 0.0033
Food Category_Prepared Food: 0.0043


We see that food categories have very less importance in the data. So we drop them

### <b> Dropping food categories

In [ ]:
threshold = 0.01

selected_features = selected_features[np.array(feature_importances) >= threshold]

print("Selected Features:", selected_features)

Selected Features: Index(['Year', 'Economic Loss (Million $)', 'Avg Waste per Capita (Kg)',
       'Population (Million)', 'Household Waste (%)'],
      dtype='object')


In [ ]:
df1 = X[selected_features]
df1.head()

,Year,Economic Loss (Million $),Avg Waste per Capita (Kg),Population (Million),Household Waste (%)
0,2023,40551.22,120.19,953.05,31.91
1,2023,36980.82,104.74,1105.47,36.06
2,2024,11196.04,136.21,1311.91,37.09
3,2024,39191.25,179.27,1349.45,68.93
4,2024,40087.26,155.55,375.84,31.16


<b>Reintroducing important columns

In [ ]:
df1["Total Waste (Tons)"]=df["Total Waste (Tons)"]

C:\Users\Souhar\AppData\Local\Temp\ipykernel_21536\3108425342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Total Waste (Tons)"]=df["Total Waste (Tons)"]


In [ ]:
df1["Country"]=df["Country"]

C:\Users\Souhar\AppData\Local\Temp\ipykernel_21536\2912594475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Country"]=df["Country"]


### <b>Grouping data to keep unique (country,year) combinations only 

In [ ]:
df1 = df1.groupby(['Country', 'Year']).mean(numeric_only=True).reset_index()


## <b>Feature Engineerng

In [ ]:
df1['Waste_Level'] = pd.qcut(df1['Total Waste (Tons)'], q=5, labels=["Very Low","Low", "Moderate", "High","Critical"])


In [ ]:
df1=df1.drop("Total Waste (Tons)",axis=1)

### <b>Encoding Waste_Level

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[["Very Low", "Low", "Moderate", "High", "Critical"]])
df1["Waste_Level_Encoded"] = encoder.fit_transform(df1[["Waste_Level"]])


In [ ]:
df1=df1.drop("Waste_Level",axis=1)

## <b> Define response and explanatory variables

In [ ]:
X=df1.drop(["Waste_Level_Encoded"],axis=1)

In [121]:
X=df1.drop(["Waste_Level_Encoded","Country"],axis=1)
y=df1["Waste_Level_Encoded"]

## <b>Scaling features

In [122]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


## <b>Train-Test Split

In [123]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X, y, df1.index, test_size=0.2, random_state=42
)

## <b>Model Fitting

### <b>SVM

In [124]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [125]:
lda = LinearDiscriminantAnalysis(n_components=2)  # Reduce to 2 dimensions
X_lda = lda.fit_transform(X_scaled, y)

# Split the reduced data
X_train_lda, X_test_lda, y_train, y_test = train_test_split(X_lda, y, test_size=0.2, random_state=42)


In [126]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

print(f"\n🔹 Training SVM with rbf kernel...")

svm_model = SVC(kernel="rbf", C=1.0, random_state=42)
svm_model.fit(X_train_lda, y_train)

y_pred = svm_model.predict(X_test_lda)

accuracy = accuracy_score(y_test, y_pred)
print(f"rbf Kernel Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))


🔹 Training SVM with rbf kernel...
rbf Kernel Accuracy: 0.7500
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.50      0.67         2
         1.0       0.67      0.67      0.67         3
         2.0       1.00      0.50      0.67         2
         3.0       0.60      1.00      0.75         3
         4.0       1.00      1.00      1.00         2

    accuracy                           0.75        12
   macro avg       0.85      0.73      0.75        12
weighted avg       0.82      0.75      0.74        12



In [127]:
import joblib
joblib.dump(svm_model, "svm_model.pkl")

['svm_model.pkl']